# Vectorized convolution operation using Numpy.
Last summer I had what I thought it was a fantastic idea: let's code a two players version of the game Asteroids, using Pyscript, and then use reinforcement learning to teach a neural network to play that game. And yes... "I thought it was a fantastic idea". Well, it wasn't that fantastic! First of all, Pyscript is a relatively new library, with hardly any documentation. Secondly (and most important), Pyscript is based in Pyodide, which doesn't support any Deep Learning library such as Keras, Tensorflow or Pytorch. And I discovered this... after two months of coding the game itself! _(you can play [here](https://bass2015.github.io/pypenth/index.html), by the way. Caution, it's still WIP)_

Being already so far along the path (I didn't want to code all the game again in another framework) I decided to keep going and implement a convolutional neural network from scratch using only Numpy. It was quite a long journey, but I learnt some useful tricks, which I hope will help other people too!

## How does a convolution work?
A convolutional neural network accepts an image as an input, and uses a series of filters or _kernels_ to extract its features. The image is basically a matrix of numbers with dimensions (channels, height, width). For example, a square RGB image of 50 px width will have a shape of (3, 50, 50). The kernels have the same number of channels, but are much smaller than the image, let's say for example 4 pixels width. Then, its shape is (3,4,4).

The convolution operation starts by "overlaying" the kernel on the top left corner of the image. Then we multiply the kernel weights by the corresponding pixels and sum everything together. This resulting number will be the top left pixel of the output matrix. Then, the kernel is _strided_ some pixels, and the operation is repeated until we have covered all the pixels in the image. We repeat this with the amount of kernels the layer has. Every 3D kernel produces a 2D matrix, so the output matrix of the layer will have as many channels as kernels. 

And what about the output width and height? In fact, it's a very simple formula:
$$Width_{out} = \frac{Width_{in} - Width_{kernel}}{Stride} +1$$

Let's say that in the first layer we have 15 kernels of size (3, 4, 4) and that we have a stride of 2. For the (3,50,50) image that we use as input, the output width and height will be $\frac{50-4}{2} + 1 = 24$, and as many channels as kernels: 15. So we'll have an output matrix of (15, 24, 24). The next layer, then, will need kernels with 15 channels. Here's an image that summarizes it a little bit. If you want more information, I've left some links in the Resources section down below. 

<div style='text-align:center'>
    <img src='https://i.stack.imgur.com/2ezvr.gif' width='500' style="border-radius: 20px">
</div>

## Basic class structure
To make a convolution, we first need a layer with the weights and biases for the kernels. It's based on this good [repository](https://github.com/gary30404/convolutional-neural-network-from-scratch-python) from [Gary](https://gary30404.github.io/). 

I won't go deep into this, the code is quite self-explanatory. Basically it sets all the parameters. As you can see, the weights are stored in a tensor with shape (num_filters, input_channels, kernel_size, kernel_size). Following with the example we saw above, our layer weights would have a shape (15, 3, 4, 4). The weights are using He initialization, in this case, becasue I originally wanted to use ReLU activation.  

In [2]:
import numpy as np

def add_to_class(Class): 
    """Decorator to add new methods to an existing class."""
    def wrapper(obj):
        setattr(Class, obj.__name__, obj)
    return wrapper

class Conv2D():
    def __init__(self, inputs_channel, num_kernels, kernel_size=4, padding=0, stride=1):
        self.num_kernels = num_kernels
        self.kernel_size = kernel_size
        self.in_channels = inputs_channel
        self.stride = stride
        self.__init_params()

    def __init_params(self):
        self.weights = np.zeros((self.num_kernels, self.in_channels, self.kernel_size, self.kernel_size), dtype=np.float32)
        self.bias = np.zeros((self.num_kernels, 1), dtype=np.float32)
        for filter in range(0,self.num_kernels):
            self.weights[filter,:,:,:] = np.random.normal(loc=0, 
                            scale=(2. / (self.in_channels * self.kernel_size * self.kernel_size)**0.5), 
                            size=(self.in_channels, self.kernel_size, self.kernel_size))

## Straight-forward solution using for loops
So, quick recap:
1. "Place" the kernel on the image
2. Multiply the kernel weights by the corresponding pixels
3. Sum everything together
4. "Stride" (or move) the kernel _x_ ammount of pixels
5. Rinse and repeat for the whole image
6. Rinse and repeat for all the kernels

Rinse and repeat... rinse and repeat... repeat... That sounds like a `for` loop, right? The most let's say "visual" solution for the problem is, indeed, nested for loops. I say visual because we can see in the code what are we doing, and it doesn't differ much from the list I wrote right above. Although, it's not the most elegant or efficient way to do it, specially if we also want to implement a batch forward method, where we have a sample of $n$ images, and we pass them all at once to the layer:

In [3]:
import time

BATCH_SIZE = 64

@add_to_class(Conv2D)
def forward(self, inputs):
        batch_size, in_channels, in_width, in_height = inputs.shape
        self.inputs = inputs 
        out_width = int((in_width - self.kernel_size) / self.stride + 1)
        out_height = int((in_height - self.kernel_size) / self.stride + 1)
        outputs = np.zeros((batch_size, self.num_kernels, out_width, out_height), dtype=np.float32)
        for b in range(batch_size):
            for k in range(self.num_kernels):
                for w in range(0, out_width, self.stride):
                    for h in range(0, out_height, self.stride):
                        outputs[b,k,w,h] = np.sum(
                                self.inputs[b, :, w:w+self.kernel_size, h:h+self.kernel_size] * \
                                self.weights[k, :, :, :]
                            ) + self.bias[k]
        return outputs

input = np.random.rand(BATCH_SIZE,3,50,50)
layer = Conv2D(3,15,4,stride=2)

In [4]:
%%timeit
layer.forward(input)

1.23 s ± 7.15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


For 64 images of 50x50 pixels (which are relatively small), the operation took 1.2 seconds in my device. And also, four nested for loops is something we should try to avoid at all costs! Let's see if we can improve it.

## What do we want?
The input for the layer is a tensor of shape (batch_size, channels, height, width), and we has kernel weights with shape (num_kernels, channels, kernel_height, kernel_weight). We want a clever way to multiply the pixel values by the kernel values, and sum everything together, but only along specific axes. For example, we want to sum the values corresponding to the the whole little cube of shape (channels, kernel_size, kernel_size), but we don't want to sum the values of the different training examples that come in the batch, or the values of the different kernels. Turns out that Numpy has a special method for that. It can take two tensors of arbitrary shape, and it allows us to choose which axes we multiply and which ones we sum. It's called `np.einsum`, and follows the [Einstein Summation Convention](https://mathworld.wolfram.com/EinsteinSummation.html) to do it. But before using it, we need to prepare a bit the input.

In the previous section, we used different `for` loops to do it. But this approximation is evidently not the best option. At the end, we're "placing" the kernels on little sections of the original image, and then moving the kernel. Let's think it the other way round: we are _dividing_ the input image in little _windows_, that we multiply and sum with the kernel weights in a special manner. So, we could arange the input to get those windows, and then do the operation. 

Let's see it more clear with a 2D matrix, easier to visualize. The matrix has a shape of (3, 3), and we're using a kernel of shape (2,2), with a stride of 1. 
<div style='text-align:center'>
    <img src='./img/strided.gif' width='250' style="border-radius: 20px">
</div>


Once we have the matrix divided in these little windows, we can multiply the kernel by them, and then sum all the values of each window, to get the final output:


<div style='text-align:center'>
    <img src='./img/stride_op.png' width='500' style="border-radius: 20px">
</div>

And, before we continue seeing how this is performed with Numpy, let's analyze the whole process:
- We have an input with shape (3, 3)
- And a kernel with shape (2, 2)
- We defined a stride of 1
- Then, the output size (following the formula above) will be $\frac{(3-2)}{1} + 1 = 2$, so it's shape will be (2, 2)
- And we see that the intermediate step, the windowed input, is a matrix with shape (2, 2) that contains other matrices with shape (2, 2). 

Let's see it with another example and different values. I recommend you to take pencil and paper and draw this matrices to visualize it better:
- Input shape - (9, 9)
- Kernel shape - (2, 2)
- Stride - 1
- Output size - $\frac{(9-2)}{1}+1=8$
- Output shape - (8, 8)
- Intermediate matrix - (8, 8, 2, 2)

Putting everything together, it's easy to see that the intermediate matrix will have a shape of _(out_height, out_width, kernel_height, kernel_width)_. This is only the case for one image with one channel, but we're dealing with a batch of multiple images with three channels each. Those values will need to be added as extra dimensions in the windowed input that is the intermediate matrix. Finally, we can say that it will have a shape of _(batch_size, channels, out_height, out_width, kernel_height, kernel_width)_.

## The magic happens, `np.lib.stride_tricks`

Now that we have the understanding of what exactly we are doing, we can see how numpy performs this operation. The first thing to look at is the attribute `strides` that the numpy arrays have:

In [9]:
input = np.arange(25).reshape(5,5)
print(input, f'\nStrides: {input.strides}')


[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]] 
Strides: (20, 4)


Personally, I was very confused by this numbers! But this [article](https://jessicastringham.net/2017/12/31/stride-tricks/) by Jessica Stringham threw some light to it. There's a detailed explanation in there, but basically, it turns out that numpy stores the matrices and tensors as long arrays of bytes. The numbers that we see above are the number of bytes that a imaginary cursor has to "jump" to get to the next value, in each dimension. In the code block above we used `np.arange`, that returns an array of integers. Integers need 4 bytes of memory to be stored. The strides (20, 4), then, means that we need 20 bytes  to jump to the next row (4 bytes x 5 elements in a row = 20 bytes), and 4 bytes to jump to the next column.
<div style='text-align:center'>
    <img src='./img/strides.png' width='500' style="border-radius: 15px">
</div>
Why are we looking at this? Because we're going to use a numpy method that needs the new strides to "make" a new array. The method is called `np.lib.stride_stricks.as_strided`, and it needs the new shape and the new strides, apart from the original array. Let's try to do with numpy what we did visually in the previous section. 

$$\begin{bmatrix}0&1&2&3&4\\5&6&7&8&9\\10&11&12&13&14\\15&16&17&18&19\\20&21&22&23&24\end{bmatrix}$$

In [ ]:
@add_to_class(Conv2D)
def __stride_input(self, inputs):
    batch_size, channels, h, w = inputs.shape
    b_stride, c_stride, h_stride, w_stride = inputs.strides
    out_w = int((w - self.kernel_size) / self.stride + 1)
    out_h = int((h - self.kernel_size) / self.stride + 1)
    new_shape = (batch_size, channels, out_h, out_w, self.kernel_size, self.kernel_size)
    new_strides = (b_stride, c_stride, self.stride * h_stride, self.stride * w_stride, h_stride, w_stride)
    return np.lib.stride_tricks.as_strided(inputs, new_shape, new_strides)


## And finally, Einstein summation

In [ ]:
@add_to_class(Conv2D)
def forward_vectorized(self, inputs):
        """Accepts four dimensional input, with shape (Batch, Channels, Height, Width)"""
        input_windows = self.__stride_input(inputs)
        self.inputs = inputs, input_windows
        output = np.einsum('bchwkt,fckt->bfhw', input_windows, self.weights) + self.bias[np.newaxis, :, np.newaxis]
        return output 

## Resources
- [Types of convolution kernels](https://towardsdatascience.com/types-of-convolution-kernels-simplified-f040cb307c37) - by [Prakar Ganesh](https://medium.com/@prakhargannu)
- [How convolutional networks work](https://stackoverflow.com/questions/54727606/how-do-convolutional-layers-cnns-work-in-keras) - Great answer in Stackoverflow by [Primusa](https://stackoverflow.com/users/8112138/primusa)
- [Build Lenet from scratch](https://github.com/gary30404/convolutional-neural-network-from-scratch-python) - repository by [Gary](https://gary30404.github.io/)
- [Xavier or He initialization?](https://github.com/christianversloot/machine-learning-articles/blob/main/he-xavier-initialization-activation-functions-choose-wisely.md) - by [Christian Versloot](https://www.linkedin.com/in/christianversloot/)
- [Stride_tricks](https://jessicastringham.net/2017/12/31/stride-tricks/) - by [Jessica Stringham](https://www.linkedin.com/in/jessicastringham/)